In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import time

print("Loading data file now, this could take a while depending on file size")
start=time.time()
df=pd.read_csv('/content/Hotel_Reviews.csv')
end=time.time()
print("Loading took"+str(round(end-start,2))+"seconds")

Loading data file now, this could take a while depending on file size
Loading took4.33seconds


In [3]:
df.describe()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,lat,lng
count,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,512470.000000,512470.000000
mean,498.081836,8.397487,18.539450,2743.743944,17.776458,7.166001,8.395077,49.442439,2.823803
std,500.538467,0.548048,29.690831,2317.464868,21.804185,11.040228,1.637856,3.466325,4.579425
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.000000,2.500000,41.328376,-0.369758
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.000000,7.500000,48.214662,-0.143372
50%,341.000000,8.400000,9.000000,2134.000000,11.000000,3.000000,8.800000,51.499981,0.010607
75%,660.000000,8.800000,23.000000,3613.000000,22.000000,8.000000,9.600000,51.516288,4.834443
max,2682.000000,9.800000,408.000000,16670.000000,395.000000,355.000000,10.000000,52.400181,16.429233


In [4]:
print("The shape of the data (rows, cols) is " + str(df.shape))

The shape of the data (rows, cols) is (515738, 17)


In [5]:
nat_freq=df["Reviewer_Nationality"].value_counts()
print("There are "+str(nat_freq.size)+" different nationalities")
print(nat_freq)

There are 227 different nationalities
Reviewer_Nationality
United Kingdom               245246
United States of America      35437
Australia                     21686
Ireland                       14827
United Arab Emirates          10235
                              ...  
Cape Verde                        1
Northern Mariana Islands          1
Tuvalu                            1
Guinea                            1
Palau                             1
Name: count, Length: 227, dtype: int64


In [6]:
print("The highest freq reviewer nationality is "+str(nat_freq.index[0]).strip()+" with "+str(nat_freq[0])+" reviews.")
print("The next top 10 are: ")
print(nat_freq[1:11].to_string())

The highest freq reviewer nationality is United Kingdom with 245246 reviews.
The next top 10 are: 
Reviewer_Nationality
United States of America     35437
Australia                    21686
Ireland                      14827
United Arab Emirates         10235
Saudi Arabia                  8951
Netherlands                   8772
Switzerland                   8678
Germany                       7941
Canada                        7894
France                        7296


/tmp/ipython-input-2939006229.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The highest freq reviewer nationality is "+str(nat_freq.index[0]).strip()+" with "+str(nat_freq[0])+" reviews.")


In [7]:
for i in nat_freq[:10].index:
  nat_df=df[df["Reviewer_Nationality"]==i]
  freq=nat_df["Hotel_Name"].value_counts()
  print("The most reviewed hotel is "+str(i).strip()+" was "+str(freq.index[0])+" reviews.")


The most reviewed hotel is United Kingdom was Britannia International Hotel Canary Wharf reviews.
The most reviewed hotel is United States of America was Hotel Esther a reviews.
The most reviewed hotel is Australia was Park Plaza Westminster Bridge London reviews.
The most reviewed hotel is Ireland was Copthorne Tara Hotel London Kensington reviews.
The most reviewed hotel is United Arab Emirates was Millennium Hotel London Knightsbridge reviews.
The most reviewed hotel is Saudi Arabia was The Cumberland A Guoman Hotel reviews.
The most reviewed hotel is Netherlands was Jaz Amsterdam reviews.
The most reviewed hotel is Switzerland was Hotel Da Vinci reviews.
The most reviewed hotel is Germany was Hotel Da Vinci reviews.
The most reviewed hotel is Canada was St James Court A Taj Hotel London reviews.


In [8]:
# First create a new dataframe based on the old one, removing the uneeded columns
hotel_freq_df = df.drop(["Hotel_Address", "Additional_Number_of_Scoring", "Review_Date", "Average_Score", "Reviewer_Nationality", "Negative_Review", "Review_Total_Negative_Word_Counts", "Positive_Review", "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given", "Reviewer_Score", "Tags", "days_since_review", "lat", "lng"], axis = 1)

# Group the rows by Hotel_Name, count them and put the result in a new column Total_Reviews_Found
hotel_freq_df['Total_Reviews_Found'] = hotel_freq_df.groupby('Hotel_Name').transform('count')

# Get rid of all the duplicated rows
hotel_freq_df = hotel_freq_df.drop_duplicates(subset = ["Hotel_Name"])
display(hotel_freq_df)

,Hotel_Name,Total_Number_of_Reviews,Total_Reviews_Found
0,Hotel Arena,1403,405
405,K K Hotel George,1831,566
971,Apex Temple Court Hotel,2619,1037
2008,The Park Grand London Paddington,4380,1770
3778,Monhotel Lounge SPA,171,35
...,...,...,...
511962,Suite Hotel 900 m zur Oper,3461,439
512401,Hotel Amadeus,717,144
512545,The Berkeley,232,100
512645,Holiday Inn London Kensington,5945,2768


In [9]:
def get_difference_review_avg(row):
  return row["Average_Score"]-row["Cal_Average_Scores"]

df['Cal_Average_Scores']=round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'),1)
df['Average_Score_Difference']=df.apply(get_difference_review_avg, axis=1)
review_scores_df=df.drop_duplicates(subset=["Hotel_Name"])
review_scores_df=review_scores_df.sort_values(by=["Average_Score_Difference"])
display(review_scores_df[["Average_Score_Difference","Average_Score","Cal_Average_Scores","Hotel_Name"]])


,Average_Score_Difference,Average_Score,Cal_Average_Scores,Hotel_Name
495945,-0.8,7.7,8.5,Best Western Hotel Astoria
111027,-0.7,8.8,9.5,Hotel Stendhal Place Vend me Paris MGallery by...
43688,-0.7,7.5,8.2,Mercure Paris Porte d Orleans
178253,-0.7,7.9,8.6,Renaissance Paris Vendome Hotel
71274,-0.5,8.9,9.4,Drawing Hotel
...,...,...,...,...
201776,0.7,7.5,6.8,Mercure Paris Op ra Faubourg Montmartre
22189,0.8,7.1,6.3,Holiday Inn Paris Montparnasse Pasteur
250308,0.9,8.6,7.7,MARQUIS Faubourg St Honor Relais Ch teaux
68936,0.9,6.8,5.9,Villa Eugenie


In [10]:
# without lambdas (using a mixture of notations to show you can use both)
start = time.time()
no_negative_reviews = sum(df.Negative_Review == "No Negative")
print("Number of No Negative reviews: " + str(no_negative_reviews))

no_positive_reviews = sum(df["Positive_Review"] == "No Positive")
print("Number of No Positive reviews: " + str(no_positive_reviews))

both_no_reviews = sum((df.Negative_Review == "No Negative") & (df.Positive_Review == "No Positive"))
print("Number of both No Negative and No Positive reviews: " + str(both_no_reviews))

end = time.time()
print("Sum took " + str(round(end - start, 2)) + " seconds")


Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Sum took 0.62 seconds
